In [2]:
import pandas as pd
import numpy as np
import pickle
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

### 데이터 합치기 - 뉴스, 블로그, 리뷰(플레이스토어 + 11번가)

#### [ 뉴스 ]

In [3]:
clova_news_df = pd.read_csv("../../data/naver_news/kakaomini_speaker_news.csv", index_col=0)
clova_news_df['source'] = 'news'
clova_news_df.head()

,title,date,publisher,url,text,source
0,카카오 미니 vs 네이버 웨이브가 고민인 당신에게,2017.10.02.,머니투데이,https://news.naver.com/main/read.nhn?mode=LSD&...,"[머니투데이 이해인 기자] [카카오 미니, 카톡 확인 및 전송 가능…네이버 웨이브,...",news
1,네이버 ‘기술’... 카카오 ‘서비스’ 양대산맥 주목,NaN,뉴스웨이,http://www.newsway.co.kr/view.php?tp=1&ud=2017...,NaN,news
2,"카카오, AI 스피커 '카카오미니C' 고음질 업데이트",2019.05.08.,블로터언론사 선정,https://news.naver.com/main/read.nhn?mode=LSD&...,카카오가 5월8일 스마트스피커 '카카오미니C'의 스피커 성능을 업데이트한다. 소프트...,news
3,스카이라이프 가입자도 AI 스피커 쓴다…대상 제품은 '구글 홈 미니',NaN,IT조선,http://it.chosun.com/site/data/html_dir/2019/0...,NaN,news
4,AI스피커 제조사 음질 경쟁…왜?,2019.05.12.,노컷뉴스언론사 선정,https://news.naver.com/main/read.nhn?mode=LSD&...,AI스피커 이용서비스 1위가 음악 감상[CBS노컷뉴스 김수영 기자]AI스피커 누구네...,news


<br/><br/>
#### [ 블로그 ]

In [4]:
clova_blog_df = pd.read_csv("../../data/naver_blog/kakaomini_speaker_blog.csv", encoding='utf-8', index_col=0)
clova_blog_df['source'] = 'blog'
clova_blog_df.head()

,title,publish_datetime,text,url,source
0,"(스크랩)로엔엔터테인먼트의 멜론, 인공지능 스피커 덕에 급성장 확실",2017-10-01 22:17:00,원문보기,http://blog.naver.com/PostView.nhn?blogId=m_co...,blog
1,대세는 인공지능 스피커! 네이버 웨이브 VS 카카오미니,2017-10-02 10:30:00,"안녕하세요. 에스메모, 앱 플레이 미뮤 등 다양한 IT Life 플랫폼 퍼블리싱 기...",http://blog.naver.com/PostView.nhn?blogId=smar...,blog
2,[2017' 칸광고제] 소셜 플랫폼의 새로운 진화. 페이스북 챗봇(Facebook ...,2017-10-02 15:36:00,NaN,http://blog.naver.com/PostView.nhn?blogId=stus...,blog
3,공감수 순위 목록 2017년 10월 02일 기준,2017-10-03 09:44:00,NaN,http://blog.naver.com/PostView.nhn?blogId=medo...,blog
4,조회수 순위 목록 2017년 10월 02일 기준,2017-10-03 09:46:00,NaN,http://blog.naver.com/PostView.nhn?blogId=7337...,blog


<br/>
#### [ 플레이스토어 리뷰 ]

In [6]:
clova_review_df = pd.read_csv('../../data/review/playstore_review_kakao_mini.csv')
clova_review_df['source'] = 'review'
clova_review_df.head()

,name,ratings,date,helpful vote,comment,source
0,Suhyun Kwon,1,2019년 5월 8일,0,빔프로젝터에 블루투스로 연결해서 스피커로 사용할때 음량 조절이 안돼요. 헤이카카오 ...,review
1,changhoon You,3,2019년 5월 5일,0,이거 해외에서는 기기위치 인식 기반 기능운 사용 할 수 없나요? 제가 베트남 호치민...,review
2,한별,3,2019년 4월 13일,4,"제가 가장 많이 사용하는 기능은 음악, 날씨, 알람 이정도 인거 같은데요 음악 많이...",review
3,김재현,5,2019년 2월 24일,53,카카오미니 잘 쓰고있습니다. 하나 바라는것이 있다면 카카오미니한테 음성명령이 아닌 ...,review
4,pp lee,1,2019년 2월 21일,26,지니 클로바 카카오 셋 중에서 제일 기능적으로 떨어지고 인식률도 후짐. 앞 두 기기...,review


<br/>
#### [ 11번가 리뷰 ]

In [7]:
_11st_clova_review_df = pd.read_csv('../../data/review/review_data_kakao.csv', index_col=0)
_11st_clova_review_df['source'] = 'review'
_11st_clova_review_df.head()

,index,nugu,score,source
0,1,여기 진짜 황당ㅡ제품과 상관없고 판매자에 대한 글입니다 아무리 최저가 비슷해도 이 ...,1,review
1,2,TV를 이용안해서 KT지니 인공지능 사용을 못한다기에 카카오 미니AI로 선택했어요!...,5,review
2,3,한달넘게 잘 사용하는 중입니다~다른 블루투스 스피커랑 다르게 라이언이 위에 올라가있...,4,review
3,4,헤이카카오라는 앱깔고 카카오 아이디로 로그인해서 사용하는거네요 음악은 어떻게 듣나 ...,5,review
4,5,물건은 꽤 빨리 받아서 만족합니다! 20일에 주문해서 23~24일에 받았던거같네요~...,3,review


<br/>
#### [ 데이터 합치기 ]


In [10]:
clova_df = pd.DataFrame({"date": pd.concat([clova_news_df.date, clova_blog_df.publish_datetime, 
                                            clova_review_df.date, pd.Series([None] * len(_11st_clova_review_df))]),
                         "texts":pd.concat([clova_news_df.text, clova_blog_df.text, 
                                            clova_review_df.comment, _11st_clova_review_df.nugu]),
                         'source':pd.concat([clova_news_df.source, clova_blog_df.source, 
                                             clova_review_df.source, _11st_clova_review_df.source]),
                         'url':pd.concat([clova_news_df.url, clova_blog_df.url, 
                                          pd.Series([None] * len(clova_review_df)), pd.Series([None] * len(_11st_clova_review_df))])
                        })

In [9]:
clova_df = clova_df[clova_df.texts.notnull()].copy()
clova_df.reset_index(inplace=True, drop=True)
clova_df

,date,texts,source
0,2017.09.04.,이미 발표된 포털업계의 소식들을 모아 한눈에 볼 수 있도록 매주 보여드리고자 합니다...,news
1,2019.05.08.,카카오가 5월8일 스마트스피커 '카카오미니C'의 스피커 성능을 업데이트한다. 소프트...,news
2,2019.05.12.,AI스피커 이용서비스 1위가 음악 감상[CBS노컷뉴스 김수영 기자]AI스피커 누구네...,news
3,2019.04.20.,‘키즈 콘텐츠’가 인공지능(AI) 스피커 시장 선점을 위한 핵심 서비스로 떠오르고 ...,news
4,NaN,"AI '눈치' 향상 팔 걷은 네이버, 질문 의도 파악·동어 반복 회피 기술 연구中성...",news
5,2019.03.27.,음성으로 채널 이동·VOD 검색 가능해져(지디넷코리아=백봉삼 기자)카카오(공동대표 ...,news
6,2019.05.13.,네이버 클로바(위)와 카카오미니. 국내 포털 양대산맥 네이버와 카카오가 차세...,news
7,2019.03.27.,"셋톱박스에 인공지능 스피커 ‘카카오미니’ 적용, 음성 명령으로 제어 전원, 채널, ...",news
8,2017.09.05.,원거리 음성 인식 위한 고성능 마이크 4개 장착 카카오의 인공지능(AI) 기반 스...,news
9,NaN,최근 국내외 정보통신기술(ICT) 기업들이 앞다퉈 인공지능(AI) 스피커를 선보이고...,news


<br/><br/>
### 텍스트 전처리

In [10]:
def clear_content(temp):
    #want_remove = ['flash 오류를 우회하기 위한 함수 추가', 'function _flash_removeCallback', 
    #               '\n', '\.\.', '[\w]+ 기자', '\xa0', '\r', '\t']
    content = re.sub('[a-zA-Z0-9]+[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', ' ', temp) # 이메일주소 없애기
    content = re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w_\.-]*)*\/?', ' ', content) # URL 없애기
    #content = re.sub('|'.join(want_remove), ' ', content)
    content = re.sub('\n|\r|\xa0', ' ', content)
    ㅠcontent = re.sub('[\(]', ' ', content) # 커머스(전자상거래)와' -> '커머스전자상거래와' 이렇게 변하던것을 -> '커머스 전자상거래'
    content = re.sub('~', ' ', content)
    #content = re.sub('[^\.\?\!\^^\♥\♡\w\d\s]','', content) # 불필요한 기호 없애기
    content = re.sub('\d', '', content)
    content = re.sub('[^\.\?\!\^^\♥\♡\w\s]','', content) # 불필요한 기호 없애기
    while '  ' in content: content = content.replace('  ', ' ') # 빈칸 없애주기 
    content = content.strip()
    return content

<br/><br/>
### 모델을 이용해서 네이버 블로그 데이터(中 clova) sentence별 긍정/부정 나누기

In [11]:
clova_df["preprocessed_text"] = clova_df['texts'].map(clear_content) 

In [12]:
clova_df.head()

,date,texts,source,preprocessed_text
0,2017.09.04.,이미 발표된 포털업계의 소식들을 모아 한눈에 볼 수 있도록 매주 보여드리고자 합니다...,news,이미 발표된 포털업계의 소식들을 모아 한눈에 볼 수 있도록 매주 보여드리고자 합니다...
1,2019.05.08.,카카오가 5월8일 스마트스피커 '카카오미니C'의 스피커 성능을 업데이트한다. 소프트...,news,카카오가 월일 스마트스피커 카카오미니C의 스피커 성능을 업데이트한다. 소프트웨어 업...
2,2019.05.12.,AI스피커 이용서비스 1위가 음악 감상[CBS노컷뉴스 김수영 기자]AI스피커 누구네...,news,AI스피커 이용서비스 위가 음악 감상CBS노컷뉴스 김수영 기자AI스피커 누구네모왼쪽...
3,2019.04.20.,‘키즈 콘텐츠’가 인공지능(AI) 스피커 시장 선점을 위한 핵심 서비스로 떠오르고 ...,news,키즈 콘텐츠가 인공지능AI 스피커 시장 선점을 위한 핵심 서비스로 떠오르고 있다. ...
4,NaN,"AI '눈치' 향상 팔 걷은 네이버, 질문 의도 파악·동어 반복 회피 기술 연구中성...",news,AI 눈치 향상 팔 걷은 네이버 질문 의도 파악동어 반복 회피 기술 연구中성장 가능...


In [13]:
del_idx = []
for num, i in enumerate(clova_df['preprocessed_text']):
    if i == '' or i == None or i == ' ':
        del_idx.append(num)
print(del_idx)

[1099, 1989]


In [14]:
clova_df.loc[clova_df.index[del_idx]]

,date,texts,source,preprocessed_text
1099,2017-11-29 12:28:00,http://blog.naver.com/shipleykorea/221150597876,blog,
1989,2018-06-15 10:24:00,http://microfocus.x-y.net/MF_Avengers.html,blog,


In [15]:
final_clova_df = clova_df.drop(clova_df.index[del_idx]).copy()

In [16]:
# 빈 데이터들을 다 삭제하고 난 뒤, 인덱스를 다시 0부터 순서대로 부여해 줌
final_clova_df.reset_index(drop=True, inplace=True)

In [17]:
final_clova_df.head()

,date,texts,source,preprocessed_text
0,2017.09.04.,이미 발표된 포털업계의 소식들을 모아 한눈에 볼 수 있도록 매주 보여드리고자 합니다...,news,이미 발표된 포털업계의 소식들을 모아 한눈에 볼 수 있도록 매주 보여드리고자 합니다...
1,2019.05.08.,카카오가 5월8일 스마트스피커 '카카오미니C'의 스피커 성능을 업데이트한다. 소프트...,news,카카오가 월일 스마트스피커 카카오미니C의 스피커 성능을 업데이트한다. 소프트웨어 업...
2,2019.05.12.,AI스피커 이용서비스 1위가 음악 감상[CBS노컷뉴스 김수영 기자]AI스피커 누구네...,news,AI스피커 이용서비스 위가 음악 감상CBS노컷뉴스 김수영 기자AI스피커 누구네모왼쪽...
3,2019.04.20.,‘키즈 콘텐츠’가 인공지능(AI) 스피커 시장 선점을 위한 핵심 서비스로 떠오르고 ...,news,키즈 콘텐츠가 인공지능AI 스피커 시장 선점을 위한 핵심 서비스로 떠오르고 있다. ...
4,NaN,"AI '눈치' 향상 팔 걷은 네이버, 질문 의도 파악·동어 반복 회피 기술 연구中성...",news,AI 눈치 향상 팔 걷은 네이버 질문 의도 파악동어 반복 회피 기술 연구中성장 가능...


- 각 문헌을 sentence별로 나누기

In [18]:
import nltk.tokenize
final_clova_df['sentence_text'] = final_clova_df['preprocessed_text'].map(nltk.tokenize.sent_tokenize)

In [19]:
final_clova_df.head()

,date,texts,source,preprocessed_text,sentence_text
0,2017.09.04.,이미 발표된 포털업계의 소식들을 모아 한눈에 볼 수 있도록 매주 보여드리고자 합니다...,news,이미 발표된 포털업계의 소식들을 모아 한눈에 볼 수 있도록 매주 보여드리고자 합니다...,[이미 발표된 포털업계의 소식들을 모아 한눈에 볼 수 있도록 매주 보여드리고자 합니...
1,2019.05.08.,카카오가 5월8일 스마트스피커 '카카오미니C'의 스피커 성능을 업데이트한다. 소프트...,news,카카오가 월일 스마트스피커 카카오미니C의 스피커 성능을 업데이트한다. 소프트웨어 업...,"[카카오가 월일 스마트스피커 카카오미니C의 스피커 성능을 업데이트한다., 소프트웨어..."
2,2019.05.12.,AI스피커 이용서비스 1위가 음악 감상[CBS노컷뉴스 김수영 기자]AI스피커 누구네...,news,AI스피커 이용서비스 위가 음악 감상CBS노컷뉴스 김수영 기자AI스피커 누구네모왼쪽...,[AI스피커 이용서비스 위가 음악 감상CBS노컷뉴스 김수영 기자AI스피커 누구네모왼...
3,2019.04.20.,‘키즈 콘텐츠’가 인공지능(AI) 스피커 시장 선점을 위한 핵심 서비스로 떠오르고 ...,news,키즈 콘텐츠가 인공지능AI 스피커 시장 선점을 위한 핵심 서비스로 떠오르고 있다. ...,[키즈 콘텐츠가 인공지능AI 스피커 시장 선점을 위한 핵심 서비스로 떠오르고 있다....
4,NaN,"AI '눈치' 향상 팔 걷은 네이버, 질문 의도 파악·동어 반복 회피 기술 연구中성...",news,AI 눈치 향상 팔 걷은 네이버 질문 의도 파악동어 반복 회피 기술 연구中성장 가능...,[AI 눈치 향상 팔 걷은 네이버 질문 의도 파악동어 반복 회피 기술 연구中성장 가...


- 나눈 sentence들을 각각 형태소 분석하기

In [20]:
# 미등록단어 지정하여 Komoran 형태소 분석 
from konlpy.tag import Komoran

#dicpath = './unregistered_dic_blog_test.txt'
#komoran = Komoran(userdic= dicpath)
dicpath = './unregistered.txt'
komoran = Komoran(userdic=dicpath)

In [21]:
final_clova_df['pos_sentence_text'] = final_clova_df['sentence_text'].map(lambda l: [komoran.pos(sentence) for sentence in l])

In [22]:
#[" ".join([wors[0] for word in sent]) for sent in final_df['pos_sentence_text']]
def pos_by_sentence(l):
    result = []
    for sent in l:
        result.append(" ".join([word[0] for word in sent]))
    return result

final_clova_df['pos_sentence_text'] = final_clova_df['pos_sentence_text'].map(pos_by_sentence)

In [ ]:
final_clova_df.head()

,date,texts,source,preprocessed_text,sentence_text,pos_sentence_text
0,2017.09.04.,이미 발표된 포털업계의 소식들을 모아 한눈에 볼 수 있도록 매주 보여드리고자 합니다...,news,이미 발표된 포털업계의 소식들을 모아 한눈에 볼 수 있도록 매주 보여드리고자 합니다...,[이미 발표된 포털업계의 소식들을 모아 한눈에 볼 수 있도록 매주 보여드리고자 합니...,[이미 발표 되 ㄴ 포털 업계 의 소식 들 을 모아 한눈 에 보 ㄹ 수 있 도록 매...
1,2019.05.08.,카카오가 5월8일 스마트스피커 '카카오미니C'의 스피커 성능을 업데이트한다. 소프트...,news,카카오가 월일 스마트스피커 카카오미니C의 스피커 성능을 업데이트한다. 소프트웨어 업...,"[카카오가 월일 스마트스피커 카카오미니C의 스피커 성능을 업데이트한다., 소프트웨어...",[카카오 가 월 이 ㄹ 스마트 스피커 카카오 미니 C 의 스피커 성능 을 업데이트 ...
2,2019.05.12.,AI스피커 이용서비스 1위가 음악 감상[CBS노컷뉴스 김수영 기자]AI스피커 누구네...,news,AI스피커 이용서비스 위가 음악 감상CBS노컷뉴스 김수영 기자AI스피커 누구네모왼쪽...,[AI스피커 이용서비스 위가 음악 감상CBS노컷뉴스 김수영 기자AI스피커 누구네모왼...,[AI 스피커 이용 서비스 위 가 음악 감상 CBS 노컷뉴스 김수영 기자 AI 스피...
3,2019.04.20.,‘키즈 콘텐츠’가 인공지능(AI) 스피커 시장 선점을 위한 핵심 서비스로 떠오르고 ...,news,키즈 콘텐츠가 인공지능AI 스피커 시장 선점을 위한 핵심 서비스로 떠오르고 있다. ...,[키즈 콘텐츠가 인공지능AI 스피커 시장 선점을 위한 핵심 서비스로 떠오르고 있다....,[키즈 콘텐츠 가 인공지능 AI 스피커 시장 선점 을 위하 ㄴ 핵심 서비스 로 떠오...
4,NaN,"AI '눈치' 향상 팔 걷은 네이버, 질문 의도 파악·동어 반복 회피 기술 연구中성...",news,AI 눈치 향상 팔 걷은 네이버 질문 의도 파악동어 반복 회피 기술 연구中성장 가능...,[AI 눈치 향상 팔 걷은 네이버 질문 의도 파악동어 반복 회피 기술 연구中성장 가...,[AI 눈치 향상 팔 ㄹ 걷 은 네이버 질문 의도 파악 동어 반복 회피 기술 연구 ...


#### sentence별 긍정(1) / 부정(-1)으로 나누기

In [ ]:
def label_by_sentence(doc):
    loaded_vectorizer = pickle.load(open('./pretrained_vectorizer.pickle', 'rb'))
    loaded_model = pickle.load(open('./pretrained_model.pickle', 'rb'))
    
    tfidf_features = loaded_vectorizer.transform(doc)
    pred_labels = loaded_model.predict(tfidf_features)
    return pred_labels

final_clova_df['label_sentence_text'] = final_clova_df['pos_sentence_text'].map(label_by_sentence)

In [ ]:
final_clova_df.to_csv("카카오.csv", encoding='utf-8')